# Lora 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
DEBUG = False
import os
if DEBUG: 
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
data_path = "content"
import torch

import urllib.request
import zipfile
from pathlib import Path
import pandas as pd
import time, math

In [3]:
from src.dataset_loader import get_enc_dataset
# Set model name
from transformers import GPT2Tokenizer
model_name = "gpt2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if DEBUG:
    device = "cpu"
print(f"Using device: {device}")


# Load GPT-2 Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Add a separate pad_token
tokenizer.pad_token = tokenizer.eos_token
#if tokenizer.pad_token is None:
#    # Use '<|PAD|>' as the padding token
#    tokenizer.add_special_tokens({'pad_token': '<|PAD|>'})
#    pad_token_id = tokenizer.pad_token_id
#    print("Added new pad_token '<|PAD|>' with ID:", pad_token_id)
batch_size = 16
max_length=None#128
train_dataset, val_dataset,test_dataset,train_loader,val_loader,test_loader, pad_token_id = get_enc_dataset(data_path,
                                                                                   tokenizer,
                                                                                   batch_size=batch_size,
                                                                                   max_length = max_length
                                                                                  )

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Index(['Text', 'Label'], dtype='object')
Max Length is:  65
Index(['Text', 'Label'], dtype='object')
Max Length is:  65
Index(['Text', 'Label'], dtype='object')
Max Length is:  65
Max Length:  65
Number of training batches: 3573, Number of validation batches: 149


## Load Petrained Model

### Test bevor Training

In [4]:
# %%

import torch.nn as nn
from transformers import GPT2ForSequenceClassification, GPT2Config
#tokenizer.pad_token = tokenizer.eos_token
from src.lora_helper import calc_accuracy,forward_for_classification

model_name = "gpt2"
num_labels = 2
model_config = GPT2Config.from_pretrained(model_name, num_labels=num_labels)
model = GPT2ForSequenceClassification.from_pretrained(model_name, config=model_config)
model.config.pad_token_id = tokenizer.pad_token_id 
model.to(device)
model.eval()

init_train_acc = calc_accuracy(train_loader, model, device, max_batches=10)
init_val_acc   = calc_accuracy(val_loader, model, device, max_batches=10)
#init_test_acc  = calc_accuracy(test_loader, model, device, max_batches=10)
init_test_acc = 0.0
print(f"Initial Accuracies -> Train: {init_train_acc*100:.2f}%, Val: {init_val_acc*100:.2f}%, Test: {init_test_acc*100:.2f}%")

2025-01-17 12:42:25.114915: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-17 12:42:25.131686: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737114145.153154   25162 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737114145.159738   25162 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-17 12:42:25.181038: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Initial Accuracies -> Train: 42.50%, Val: 48.12%, Test: 0.00%


## Different Strategy

### Strategy A: Full Fine-Tuning

In [5]:
# %%
from src.lora_train import train_model_full_finetune
import copy
import time
import torch 
import torch.nn as nn
print("=== Strategy A: Full Fine-Tuning ===")
modelA = copy.deepcopy(model)
full_tune_params = sum(p.numel() for p in modelA.parameters() if p.requires_grad)
print(f"[Full Fine-Tuning] Trainable Params: {full_tune_params}")

init_accA = calc_accuracy(train_loader, modelA, device, max_batches=10)
print(f"[Full Fine-Tuning] Initial Train Acc (first 10 batches): {init_accA*100:.2f}%")

elapsedA, ft_train_losses, ft_val_accs = train_model_full_finetune(
    modelA, train_loader, val_loader, device, epochs=5, lr=5e-5
)

train_accA = calc_accuracy(train_loader, modelA, device)
val_accA   = calc_accuracy(val_loader, modelA, device)
test_accA  = calc_accuracy(test_loader, modelA, device)
print(f"[Full Fine-Tuning] Time: {elapsedA:.2f}s, TrainAcc={train_accA*100:.2f}%, ValAcc={val_accA*100:.2f}%, TestAcc={test_accA*100:.2f}%\n")

=== Strategy A: Full Fine-Tuning ===
[Full Fine-Tuning] Trainable Params: 124441344
[Full Fine-Tuning] Initial Train Acc (first 10 batches): 44.38%
Epoch 1/5, step 500/3573, loss = 0.2763
Epoch 1/5, step 1000/3573, loss = 0.4824
Epoch 1/5, step 1500/3573, loss = 0.1989
Epoch 1/5, step 2000/3573, loss = 0.2015
Epoch 1/5, step 2500/3573, loss = 0.3645
Epoch 1/5, step 3000/3573, loss = 0.1001
Epoch 1/5, step 3500/3573, loss = 0.3682
Epoch 1/5, step 3570/3573, loss = 0.1308Epoch=1, Loss=0.2701, ValAcc=92.49%
Epoch 2/5, step 500/3573, loss = 0.0914
Epoch 2/5, step 1000/3573, loss = 0.0514
Epoch 2/5, step 1500/3573, loss = 0.0654
Epoch 2/5, step 2000/3573, loss = 0.1347
Epoch 2/5, step 2500/3573, loss = 0.0311
Epoch 2/5, step 3000/3573, loss = 0.0437
Epoch 2/5, step 3500/3573, loss = 0.1943
Epoch 2/5, step 3570/3573, loss = 0.0254Epoch=2, Loss=0.1501, ValAcc=93.54%
Epoch 3/5, step 500/3573, loss = 0.1641
Epoch 3/5, step 1000/3573, loss = 0.1421
Epoch 3/5, step 1500/3573, loss = 0.3240
Epoch 

## Strategy B LoRa

In [8]:
from src.lora_model import *
from src.lora_train import train_model_lora
import copy
import time

In [9]:
# %%


print("=== Strategy B: LoRA ===")

modelB = copy.deepcopy(model)  # Duplicate the base model
replace_modules_with_lora(modelB, rank=16, alpha=32)  # Replace layers
modelB.to(device)

elapsedB, lora_train_losses, lora_val_accs = train_model_lora(
    modelB, train_loader, val_loader, device, epochs=5, lr=1e-4, log_grad_norms=True
)

def calc_accuracy_full(loader, model, device, max_batches=None):
    model.eval()
    correct, total = 0, 0
    for i, (input_ids, attention_mask, y_batch) in enumerate(loader):
        if max_batches and (i+1) > max_batches:
            break
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        y_batch = y_batch.to(device)
        with torch.no_grad():
            logits = forward_for_classification(model, input_ids, attention_mask, device)
        preds = torch.argmax(logits, dim=-1)
        correct += (preds == y_batch).sum().item()
        total   += y_batch.size(0)
    return correct / total if total > 0 else 0

train_accB = calc_accuracy_full(train_loader, modelB, device)
val_accB   = calc_accuracy_full(val_loader, modelB, device)
test_accB  = calc_accuracy_full(test_loader, modelB, device)
print(f"[LoRA] Training Time: {elapsedB:.2f}s")
print(f"[LoRA] TrainAcc={train_accB*100:.2f}%, ValAcc={val_accB*100:.2f}%, TestAcc={test_accB*100:.2f}%\n")

=== Strategy B: LoRA ===
[LoRALayer] in_dim=768, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=3072, rank=16, alpha=32
[LoRALayer] in_dim=3072, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=3072, rank=16, alpha=32
[LoRALayer] in_dim=3072, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=3072, rank=16, alpha=32
[LoRALayer] in_dim=3072, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=3072, rank=16, alpha=32
[LoRALayer] in_dim=3072, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=3072, rank=16, alpha=32
[LoRALayer] in_dim=3072, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=768, rank=16, alpha=32
[LoRALayer] in_dim=768, out_dim=3072, rank=16, alpha=32
[LoRALayer] in_dim=3072, out_

/mnt/vast-react/home/joris.hellwig/u13685/jupyterhub-gwdg/ex3/src/lora_train.py:75: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
/mnt/vast-react/home/joris.hellwig/u13685/jupyterhub-gwdg/ex3/src/lora_train.py:95: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Epoch 1/5, step 500/3573, loss = 0.6137
Epoch 1/5, step 1000/3573, loss = 0.6961
Epoch 1/5, step 1500/3573, loss = 0.4717
Epoch 1/5, step 2000/3573, loss = 0.8589
Epoch 1/5, step 2500/3573, loss = 0.5421
Epoch 1/5, step 3000/3573, loss = 0.7865
Epoch 1/5, step 3500/3573, loss = 0.4068
Epoch 1/5, step 3570/3573, loss = 0.6336Epoch=1, Loss=0.5694, ValAcc=79.73%
Epoch 2/5, step 500/3573, loss = 0.6780
Epoch 2/5, step 1000/3573, loss = 0.5363
Epoch 2/5, step 1500/3573, loss = 0.6228
Epoch 2/5, step 2000/3573, loss = 0.2439
Epoch 2/5, step 2500/3573, loss = 0.5157
Epoch 2/5, step 3000/3573, loss = 0.6061
Epoch 2/5, step 3500/3573, loss = 0.3295
Epoch 2/5, step 3570/3573, loss = 0.4837Epoch=2, Loss=0.4618, ValAcc=82.58%
Epoch 3/5, step 500/3573, loss = 0.3901
Epoch 3/5, step 1000/3573, loss = 0.3308
Epoch 3/5, step 1500/3573, loss = 0.3026
Epoch 3/5, step 2000/3573, loss = 0.6323
Epoch 3/5, step 2500/3573, loss = 0.4690
Epoch 3/5, step 3000/3573, loss = 0.2564
Epoch 3/5, step 3500/3573, loss

ValueError: too many values to unpack (expected 3)

 #### B.3: Advanced Metrics (Precision, Recall, F1)

 We'll calculate a more comprehensive set of metrics on the LoRA model to evaluate performance beyond accuracy.

In [ ]:
# %%
from src.lora_helper import advanced_metrics
accB, precB, recB, f1B = advanced_metrics(test_loader, modelB, device)
print(f"[LoRA Advanced Metrics on Test] Accuracy={accB*100:.2f}%, Precision={precB*100:.2f}%, Recall={recB*100:.2f}%, F1={f1B*100:.2f}%")